# 구글 크롤링

In [1]:
#pip install selenium

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import urllib.request
import os

In [ ]:
from selenium.webdriver.chrome.service import Service  # Service 클래스를 import
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

# Chrome 옵션 설정
options = Options()
options.add_argument('--start-maximized')  # 창 최대화
options.add_argument('--disable-infobars')  # 정보바 비활성화
options.add_argument('--disable-extensions')  # 확장 기능 비활성화

# Chrome Driver 경로 지정 및 서비스 객체 생성
s = Service(r"D:/Project01/selenium/chromedriver.exe")

# 드라이버 초기화
driver = webdriver.Chrome(service=s, options=options)
driver.implicitly_wait(3)

# 검색하고자 하는 음료 이름
beverages = ['cocacola', 'sprite', 'fanta']
# 저장할 기본 경로
base_dir = "D:/Project01/Project02_data/normal_data/"

for beverage in beverages:
    # 해당 음료 이미지 저장 폴더 생성 (폴더가 없다면)
    save_folder = os.path.join(base_dir, beverage)
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    # 구글 이미지 검색 페이지 접근
    driver.get("https://www.google.co.kr/imghp?hl=ko&ogbl")
    elem = driver.find_element(By.NAME, "q") # 최신 selenium버전으로 수정
    elem.send_keys(f"{beverage} can")  # 음료 이름으로 검색
    elem.send_keys(Keys.RETURN)

    # 스크롤 다운 코드 개선
    SCROLL_PAUSE_TIME = 1
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # 이미지 다운로드
    images = driver.find_elements(By.CSS_SELECTOR, ".rg_i.Q4LuWd")
    count = 0
    for image in images:
        try:
            image.click()
            time.sleep(2) 
            imgUrl = driver.find_element(By.CSS_SELECTOR, '.sFlh5c.pT0Scc.iPVvYb').get_attribute("src")
            opener = urllib.request.build_opener()
            opener.addheaders = [('User-Agent', 'Mozilla/5.0')]
            urllib.request.install_opener(opener)
            time.sleep(2) 
            image_path = os.path.join(save_folder, f"{count}.jpg")
            urllib.request.urlretrieve(imgUrl, image_path)
            print(f"{count} {beverage} image download success")
            count += 1
            if count > 100:
                break
        except Exception as e:
            print(e)  # 예외 정보 출력

print("Crawling and downloading images completed.")
driver.quit()

0 cocacola image download success
1 cocacola image download success
2 cocacola image download success
3 cocacola image download success
4 cocacola image download success
5 cocacola image download success
6 cocacola image download success
7 cocacola image download success
8 cocacola image download success
9 cocacola image download success
10 cocacola image download success
11 cocacola image download success
Message: no such element: Unable to locate element: {"method":"css selector","selector":".sFlh5c.pT0Scc.iPVvYb"}
  (Session info: chrome=125.0.6422.113); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF696A61F22+60322]
	(No symbol) [0x00007FF6969DCE99]
	(No symbol) [0x00007FF696897EBA]
	(No symbol) [0x00007FF6968E7676]
	(No symbol) [0x00007FF6968E773C]
	(No symbol) [0x00007FF69692E967]
	(No symbol) [0x00007FF69690C25F]
	(No symbol) [0x00007FF69692B

In [4]:
#pip install opencv-python

In [6]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Flatten, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from PIL import Image
import io
import os
from glob import glob
import warnings
warnings.filterwarnings(action='ignore')
import tensorflow as tf
import random
import cv2 as cv

coke_list=glob('D:/Project01/Project02_data/normal_data/cocacola/*.jpg')
print(len(coke_list))   # 212
sprite_list= glob('D:/Project01/Project02_data/normal_data/sprite/*.jpg')
print(len(sprite_list))   # 212
fanta_list= glob('D:/Project01/Project02_data/normal_data/fanta/*.jpg')
print(len(fanta_list))   # 212

####################

name_list = [coke_list, sprite_list, fanta_list]
name = ['cocacola', 'sprite', 'fanta']

def labeling (image_list, label, data_list, label_list) :
    for i in image_list :
        try :
            img = cv.imread(i)
            img = cv.resize(img,(64,64))
            # cv.imshow("image", img)
            # cv.waitKey(0)
            img=np.array(img)/255.
            data_list.append(img)
            label_list.append(label)
        except :
            pass
    return data_list, label_list


index = 0
for std in name_list :
    print("index : ", index)
    print("standard : ", name[index])

    # 기준이 되는 음료수 데이터를 넣는다.
    std_data = list()
    std_label = list()

    copy_list = name_list.copy()
    copy_name = name.copy()

    std_data, std_label = labeling (std, 0, std_data, std_label)     # labeling 0 : 기준이 되는 음료수와 동일하다.

    std_data = np.array(std_data)
    std_label = np.array(std_label)


    # 기준 음료수 외 나머지를 하나의 리스트에 넣는다.
    copy_list.remove(std)
    copy_name.remove(copy_name[index])

    dif_data = list()
    dif_label = list()

    for dif in copy_list :
        dif = random.sample(dif, len(dif)//5)                        # 랜덤 추출 : 기준이 아닌 음료수에서는 각각 20% 개수씩 가져온다.
        dif_data, dif_label = labeling (dif, 1, dif_data, dif_label) # labeling 1: 기준이 되는 음료수와 다르다.

    dif_data = np.array(dif_data)
    dif_label = np.array(dif_label)

    # 기준 데이터의 자동 변수 생성
    globals()['std_data_{}'.format(name[index])] = std_data
    globals()['std_label_{}'.format(name[index])] = std_label

    # 비교 데이터의 자동 변수 생성
    globals()['dif_data_{}'.format(name[index])] = dif_data
    globals()['dif_label_{}'.format(name[index])] = dif_label

    index += 1

    # print("std data", len(std_data))
    # print("dif data", len(dif_data))

print("==========자동 변수 생성 확인=========")
print(" std_data|std_label|dif_data|dif_label")
print(len(std_data_cocacola),len(std_label_cocacola),len(dif_data_cocacola),len(dif_label_cocacola))
print(len(std_data_sprite),len(std_label_sprite),len(dif_data_sprite),len(dif_label_sprite))
print(len(std_data_fanta),len(std_label_fanta),len(dif_data_fanta),len(dif_label_fanta))


# np.save =====================================================================

#print("np.save start>>>>>>>>>>>>>>>>>>>>>")
#np.save('../Project02_data/9.npy/cv_std_data_cocacola', arr=std_data_cocacola)
#np.save('../Project02_data/9.npy/cv_std_label_cocacola', arr=std_label_cocacola)
#np.save('../Project02_data/9.npy/cv_dif_data_cocacola', arr=dif_data_cocacola)
#np.save('../Project02_data/9.npy/cv_dif_label_cocacola', arr=dif_label_cocacola)

#np.save('../Project02_data/9.npy/std_data_sprite', arr=std_data_sprite)
#np.save('../Project02_data/9.npy/std_label_sprite', arr=std_label_sprite)
#np.save('../Project02_data/9.npy/dif_data_sprite', arr=dif_data_sprite)
#np.save('../Project02_data/9.npy/dif_label_sprite', arr=dif_label_sprite)
#print("np.save end<<<<<<<<<<<<<<<<<<<<<<<")

# np.save =====================================================================

import os

# 저장할 디렉토리 경로
save_dir = 'D:/Project01/Project02_data/9.npy/'

# 디렉토리가 존재하는지 확인하고, 없으면 생성
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

print("np.save start>>>>>>>>>>>>>>>>>>>>>")
np.save(os.path.join(save_dir, 'cv_std_data_cocacola.npy'), arr=std_data_cocacola)
np.save(os.path.join(save_dir, 'cv_std_label_cocacola.npy'), arr=std_label_cocacola)
np.save(os.path.join(save_dir, 'cv_dif_data_cocacola.npy'), arr=dif_data_cocacola)
np.save(os.path.join(save_dir, 'cv_dif_label_cocacola.npy'), arr=dif_label_cocacola)

np.save(os.path.join(save_dir, 'std_data_sprite.npy'), arr=std_data_sprite)
np.save(os.path.join(save_dir, 'std_label_sprite.npy'), arr=std_label_sprite)
np.save(os.path.join(save_dir, 'dif_data_sprite.npy'), arr=dif_data_sprite)
np.save(os.path.join(save_dir, 'dif_label_sprite.npy'), arr=dif_label_sprite)

np.save(os.path.join(save_dir, 'std_data_fanta.npy'), arr=std_data_fanta)
np.save(os.path.join(save_dir, 'std_label_fanta.npy'), arr=std_label_fanta)
np.save(os.path.join(save_dir, 'dif_data_fanta.npy'), arr=dif_data_fanta)
np.save(os.path.join(save_dir, 'dif_label_fanta.npy'), arr=dif_label_fanta)
print("np.save end<<<<<<<<<<<<<<<<<<<<<<<")

10
10
10
index :  0
standard :  cocacola
index :  1
standard :  sprite
index :  2
standard :  fanta
==========자동 변수 생성 확인=========
 std_data|std_label|dif_data|dif_label
10 10 4 4
10 10 4 4
10 10 4 4
np.save start>>>>>>>>>>>>>>>>>>>>>
np.save end<<<<<<<<<<<<<<<<<<<<<<<


In [10]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Flatten, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from PIL import Image
import io
import os
from glob import glob
import warnings
warnings.filterwarnings(action='ignore')
import tensorflow as tf
from tensorflow.keras import backend as K

# 참고사이트 : https://buillee.tistory.com/111
# https://github.com/skanwngud/Project/blob/main/mask/human_nohuman.py

coke_list=glob('D:/Project01/Project02_data/normal_data/cocacola/*.jpg')
print(len(coke_list))   # 212
sprite_list= glob('D:/Project01/Project02_data/normal_data/sprite/*.jpg')
print(len(sprite_list))   # 212
fanta_list= glob('D:/Project01/Project02_data/normal_data/fanta/*.jpg')
print(len(fanta_list))   # 212

#################### 

name_list = [coke_list, sprite_list, fanta_list]
name = ['cocacola', 'sprite', 'fanta']

train_datagen = ImageDataGenerator(
    width_shift_range=0.2,
    height_shift_range=0.2,
    rotation_range=5,
    zoom_range=0.5,
    fill_mode='nearest',
)

etc_datagen = ImageDataGenerator()

# 1. DATA
# np.load
x_train = np.load('D:/Project01/Project02_data/9.npy/cv_color_x_train_cocacola.npy')
x_test = np.load('D:/Project01/Project02_data/9.npy/cv_color_x_test_cocacola.npy')
x_valid = np.load('D:/Project01/Project02_data/9.npy/cv_color_x_valid_cocacola.npy')
y_train = np.load('D:/Project01/Project02_data/9.npy/cv_color_y_train_cocacola.npy')
y_test = np.load('D:/Project01/Project02_data/9.npy/cv_color_y_test_cocacola.npy')
y_valid = np.load('D:/Project01/Project02_data/9.npy/cv_color_y_valid_cocacola.npy')

print("x : ", x_train.shape, x_test.shape, x_valid.shape)
print("y : ", y_train.shape, y_test.shape, y_valid.shape)

# x :  (341, 64, 64, 3) (43, 64, 64, 3) (38, 64, 64, 3)
# y :  (341,) (43,) (38,)

batch = 16
train_generator = train_datagen.flow(x_train, y_train, batch_size=batch)
test_generator = etc_datagen.flow(x_test, y_test, batch_size=batch)
valid_generator = etc_datagen.flow(x_valid, y_valid)

# 2. Modeling
def modeling():
    model = Sequential()
    model.add(Conv2D(32, 3, padding='same', activation='elu', input_shape=(x_train.shape[1], x_train.shape[2], x_train.shape[3])))
    model.add(BatchNormalization())
    model.add(Conv2D(32, 3, padding='same', activation='elu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(2))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, 2, padding='same', activation='elu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, 2, padding='same', activation='elu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(3))
    model.add(Dropout(0.3))

    model.add(Conv2D(128, 2, padding='same', activation='elu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, 2, padding='same', activation='elu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(2))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(128, activation='elu'))
    model.add(Dense(32, activation='elu'))
    model.add(Dense(1, activation='sigmoid'))

    return model

model = modeling()

def recall_m(y_true, y_pred):
    y_true = K.cast(y_true, 'float32')
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    y_true = K.cast(y_true, 'float32')
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

# 3. Compile, train
es = EarlyStopping(monitor='val_loss', patience=20, mode='min')
lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=10, mode='min')
path = 'D:/Project01/Project02_data/9.cp/find_cocacola_{val_loss:.4f}.keras'  # .keras 확장자로 변경
cp = ModelCheckpoint(path, monitor='val_loss', mode='min', save_best_only=True)

model.compile(optimizer='adam', metrics=['acc', f1_m], loss='binary_crossentropy')
hist = model.fit(
    train_generator, steps_per_epoch=len(x_train) // batch, epochs=100, validation_data=valid_generator, callbacks=[es, lr, cp])

# 4. Evaluate, Predict
loss, acc, f1_score = model.evaluate(test_generator)
print("loss : ", loss)
print("acc : ", acc)
print("f1_score : ", f1_score)

# loss :  0.0006527779041789472
# acc :  1.0
# f1_score :  1.0

10
10
10
x :  (10, 64, 64, 3) (2, 64, 64, 3) (2, 64, 64, 3)
y :  (10,) (2,) (2,)
Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - acc: 0.8000 - f1_m: 0.0000e+00 - loss: 0.4850 - val_acc: 0.5000 - val_f1_m: 1.3333 - val_loss: 0.8219 - learning_rate: 0.0010
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step - acc: 0.2000 - f1_m: 3.3333 - loss: 3.0607 - val_acc: 0.5000 - val_f1_m: 1.3333 - val_loss: 0.6875 - learning_rate: 0.0010
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step - acc: 0.8000 - f1_m: 0.0000e+00 - loss: 0.9219 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_loss: 0.6359 - learning_rate: 0.0010
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step - acc: 0.8000 - f1_m: 0.0000e+00 - loss: 1.4977 - val_acc: 0.5000 - val_f1_m: 0.0000e+00 - val_loss: 0.6316 - learning_rate: 0.0010
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step - acc: 0.8000 - f1_m: 0.0000e+00 - loss: 0.5311 - val_acc: 0.5000 - val_f1_m: 0.0000e+00 - val_loss: 0.6252 - learning_rate: 0.0010
Epoch 6/100
1/1 ━━━━━━━

텐서플로우 ver 1.x: from tensorflow.keras.wrappers.scikit_learn import KerasClassifier <br>
텐서플로우 ver 2.x: from scikeras.wrappers import KerasClassifier

## 오류 발생하는 코드

In [9]:
pip install scikeras

Note: you may need to restart the kernel to use updated packages.


In [34]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Flatten, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from PIL import Image
import io
import os
from glob import glob
import warnings
warnings.filterwarnings(action='ignore')
import tensorflow as tf
from scikeras.wrappers import KerasClassifier

coke_list = glob('D:/Project01/Project02_data/normal_data/cocacola/*.jpg')
print(len(coke_list))  # 212
sprite_list = glob('D:/Project01/Project02_data/normal_data/sprite/*.jpg')
print(len(sprite_list))  # 212

#################### 

name_list = [coke_list, sprite_list]
name = ['cocacola','sprite']

train_datagen = ImageDataGenerator(
    # rescale=1./255,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rotation_range=5,
    zoom_range=0.5,
    fill_mode='nearest',
)

etc_datagen = ImageDataGenerator()

# 1. DATA
# np.load
x_train = np.load('D:/Project01/Project02_data/9.npy/cv_color_x_train_cocacola.npy')
x_test = np.load('D:/Project01/Project02_data/9.npy/cv_color_x_test_cocacola.npy')
x_valid = np.load('D:/Project01/Project02_data/9.npy/cv_color_x_valid_cocacola.npy')
y_train = np.load('D:/Project01/Project02_data/9.npy/cv_color_y_train_cocacola.npy')
y_test = np.load('D:/Project01/Project02_data/9.npy/cv_color_y_test_cocacola.npy')
y_valid = np.load('D:/Project01/Project02_data/9.npy/cv_color_y_valid_cocacola.npy')

print("x : ", x_train.shape, x_test.shape, x_valid.shape)  
print("y : ", y_train.shape, y_test.shape, y_valid.shape)

batch = 16
train_generator = train_datagen.flow(x_train, y_train, batch_size=batch)
test_generator = etc_datagen.flow(x_test, y_test, batch_size=batch)
valid_generator = etc_datagen.flow(x_valid, y_valid)

#2 Modeling
def build_model(optimizer='adam', drop1=0.2, drop2=0.3, drop3=0.4, kernel1=3, kernel2=2, kernel3=2, pool1=2, pool2=2, pool3=2, node1=128, node2=64, activation1='relu', activation2='relu'):
# def modeling(optimizer='adam', drop2=0.3, drop3=0.4, kernel1=2, kernel2=2, kernel3=2) :
# def modeling(pool1=2, pool2=2, pool3=2) :
    model = Sequential()
    model.add(Conv2D(32, kernel1, padding='same', activation=activation1, \
        input_shape=(x_train.shape[1], x_train.shape[2], x_train.shape[3])))
    model.add(BatchNormalization())
    model.add(Conv2D(32, kernel1, padding='same', activation=activation1))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool1))
    model.add(Dropout(drop1))

    model.add(Conv2D(64, kernel2, padding='same', activation=activation1))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel2, padding='same', activation=activation1))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool2))
    model.add(Dropout(drop2))

    model.add(Conv2D(128, kernel3, padding='same', activation=activation1))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel3, padding='same', activation=activation1))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool3))
    model.add(Dropout(drop3))

    model.add(Flatten())
    model.add(Dense(node1, activation=activation2))
    model.add(Dense(node2, activation=activation2))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model


# model = modeling()

# hyperparameter 확인
model = KerasClassifier(model=build_model, batch_size=16, epochs=10)   


# 하이퍼파라미터 생성 함수
def create_hyperparameters():
    return {
        "model__optimizer": ['rmsprop', 'adam'],
        "model__drop1": [0.2, 0.3],
        "model__drop2": [0.2, 0.3],
        "model__drop3": [0.2, 0.3],
        "model__kernel1": [2, 3],
        "model__kernel2": [2, 3],
        "model__kernel3": [2, 3],
        "model__pool1": [2, 3],
        "model__pool2": [2, 3],
        "model__pool3": [2, 3],
        "model__node1": [128, 256],
        "model__node2": [64, 128],
        "model__activation1": ['relu', 'elu'],
        "model__activation2": ['relu', 'sigmoid']
    }

hyperparameters = create_hyperparameters()
search = RandomizedSearchCV(model, hyperparameters, cv=3, n_iter=10)
search.fit(x_train, y_train)  # x_train, y_train는 적절한 데이터셋으로 가정
print("best_params : ", search.best_params_)
# best_params :  {'node2': 32, 'node1': 128, 'drop3': 0.2, 'drop2': 0.3, 'drop1': 0.2, \
# 'batch_size': 16, 'activation2': 'elu', 'activation1': 'elu'}
# best_params :  {'optimizer': 'adam', 'kernel3': 2, 'kernel2': 2, 'kernel1': 3, 'drop3': 0.3, 'drop2': 0.2}
# best_params :  {'pool3': 2, 'pool2': 3, 'pool1': 2}

10
10
x :  (9, 64, 64, 3) (2, 64, 64, 3) (1, 64, 64, 3)
y :  (9,) (2,) (1,)
Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.5000 - loss: 0.7103
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.8333 - loss: 0.6020
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.8333 - loss: 0.4745
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.8333 - loss: 0.4157
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - accuracy: 0.8333 - loss: 0.3586
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - accuracy: 0.8333 - loss: 0.2856
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - accuracy: 0.8333 - loss: 0.2158
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 1.0000 - loss: 0.1160
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 1.0000 - loss: 0.0741
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 1.0000 - loss: 0.0581
Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.8333 - loss: 0.6297
Epoch 2/10
1/

ValueError: Could not interpret metric identifier: loss

## 오류는 안나는데 기존코드와 같은 알고리즘이 맞는지 모르겠다..?

In [4]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Flatten, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from PIL import Image
import io
import os
from glob import glob
import warnings
warnings.filterwarnings(action='ignore')
import tensorflow as tf
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV

coke_list = glob('D:/Project01/Project02_data/normal_data/cocacola/*.jpg')
print(len(coke_list))  # 212
sprite_list = glob('D:/Project01/Project02_data/normal_data/sprite/*.jpg')
print(len(sprite_list))  # 212

#################### 

name_list = [coke_list, sprite_list]
name = ['cocacola','sprite']

train_datagen = ImageDataGenerator(
    # rescale=1./255,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rotation_range=5,
    zoom_range=0.5,
    fill_mode='nearest',
)

etc_datagen = ImageDataGenerator()

# 1. DATA
# np.load
x_train = np.load('D:/Project01/Project02_data/9.npy/cv_color_x_train_cocacola.npy')
x_test = np.load('D:/Project01/Project02_data/9.npy/cv_color_x_test_cocacola.npy')
x_valid = np.load('D:/Project01/Project02_data/9.npy/cv_color_x_valid_cocacola.npy')
y_train = np.load('D:/Project01/Project02_data/9.npy/cv_color_y_train_cocacola.npy')
y_test = np.load('D:/Project01/Project02_data/9.npy/cv_color_y_test_cocacola.npy')
y_valid = np.load('D:/Project01/Project02_data/9.npy/cv_color_y_valid_cocacola.npy')

print("x : ", x_train.shape, x_test.shape, x_valid.shape)  
print("y : ", y_train.shape, y_test.shape, y_valid.shape)

batch = 16
train_generator = train_datagen.flow(x_train, y_train, batch_size=batch)
test_generator = etc_datagen.flow(x_test, y_test, batch_size=batch)
valid_generator = etc_datagen.flow(x_valid, y_valid)

#2 Modeling
def modeling(drop1=0.2, node1=128, node2=64, activation1='relu', activation2='relu'):
    def modeling(optimizer='adam', drop2=0.3, drop3=0.4, kernel1=2, kernel2=2, kernel3=2):
        def modeling(pool1=2, pool2=2, pool3=2):
            model = Sequential()
            model.add(Conv2D(32, 3, padding='same', activation=activation1, input_shape=(x_train.shape[1], x_train.shape[2], x_train.shape[3])))
            model.add(BatchNormalization())
            model.add(Conv2D(32, 3, padding='same', activation=activation1))
            model.add(BatchNormalization())
            model.add(MaxPool2D(pool1))
            model.add(Dropout(drop1))
        
            model.add(Conv2D(64, 2, padding='same', activation=activation1))
            model.add(BatchNormalization())
            model.add(Conv2D(64, 2, padding='same', activation=activation1))
            model.add(BatchNormalization())
            model.add(MaxPool2D(pool2))
            model.add(Dropout(drop2))
        
            model.add(Conv2D(128, 2, padding='same', activation=activation1))
            model.add(BatchNormalization())
            model.add(Conv2D(128, 2, padding='same', activation=activation1))
            model.add(BatchNormalization())
            model.add(MaxPool2D(pool3))
            model.add(Dropout(drop3))
        
            model.add(Flatten())
            model.add(Dense(node1, activation=activation2))
            model.add(Dense(node2, activation=activation2))
            model.add(Dense(1, activation='sigmoid'))
        
            model.compile(optimizer='adam', metrics=['accuracy'], loss='binary_crossentropy')
            return model
model = modeling()

10
10
x :  (9, 64, 64, 3) (2, 64, 64, 3) (1, 64, 64, 3)
y :  (9,) (2,) (1,)


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Flatten, BatchNormalization
from tensorflow.keras.optimizers import Adam

input_shape = (x_train.shape[1], x_train.shape[2], x_train.shape[3])

def build_model():
    model = Sequential()
    
    # Conv Block 1
    model.add(Conv2D(32, (3, 3), padding='same', activation='elu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3, 3), padding='same', activation='elu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D((2, 2)))
    model.add(Dropout(0.2))
    
    # Conv Block 2
    model.add(Conv2D(64, (2, 2), padding='same', activation='elu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (2, 2), padding='same', activation='elu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D((3, 3)))
    model.add(Dropout(0.3))
    
    # Conv Block 3
    model.add(Conv2D(128, (2, 2), padding='same', activation='elu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (2, 2), padding='same', activation='elu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D((2, 2)))
    model.add(Dropout(0.3))
    
    # Fully Connected Layers
    model.add(Flatten())
    model.add(Dense(128, activation='elu'))
    model.add(Dense(32, activation='elu'))
    model.add(Dense(1, activation='sigmoid'))  # 이진 분류
    
    # 모델 컴파일
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# 모델 생성
model = build_model()

# 모델 요약 출력
model.summary()

# 모델 훈련
history = model.fit(train_generator, validation_data=valid_generator, epochs=50, batch_size=16)

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)                    │ (None, 64, 64, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (None, 64, 64, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 64, 64, 32)          │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 64, 64, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 32, 32, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 32, 32, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (None, 32, 32, 64)          │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_8                │ (None, 32, 32, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ (None, 32, 32, 64)          │          16,448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_9                │ (None, 32, 32, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 10, 10, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 10, 10, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_10 (Conv2D)                   │ (None, 10, 10, 128)         │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_10               │ (None, 10, 10, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_11 (Conv2D)                   │ (None, 10, 10, 128)         │          65,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_11               │ (None, 10, 10, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 5, 5, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 5, 5, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 549,089 (2.09 MB)

 Trainable params: 548,193 (2.09 MB)

 Non-trainable params: 896 (3.50 KB)

Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - accuracy: 0.7778 - loss: 0.4733 - val_accuracy: 1.0000 - val_loss: 0.2587
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step - accuracy: 0.8889 - loss: 1.3200 - val_accuracy: 1.0000 - val_loss: 0.3164
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step - accuracy: 0.8889 - loss: 0.4285 - val_accuracy: 1.0000 - val_loss: 0.5048
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step - accuracy: 0.7778 - loss: 0.2616 - val_accuracy: 1.0000 - val_loss: 0.3857
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - accuracy: 0.8889 - loss: 0.1528 - val_accuracy: 1.0000 - val_loss: 0.2073
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step - accuracy: 0.8889 - loss: 0.1447 - val_accuracy: 1.0000 - val_loss: 0.1195
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - accuracy: 1.0000 - loss: 0.0015 - val_accuracy: 1.0000 - val_loss: 0.0673
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step - accuracy: 1.0000 - loss: 1.8758e-04 - val_accuracy: 1.0000 - val_loss